# Prompt

##### 2. Bernstein-Vazirani Algorithm (for n = 4) The Bernstein-Vazirani algorithm is used to determine a hidden string s ∈ {0, 1} n given an oracle that computes f(x) = s · x mod 2, where s · x represents the bitwise inner product. Construct a Qiskit circuit for the Bernstein-Vazirani algorithm with 4 input qubits and 1 ancillary qubit. Assume the hidden string is s = 1011. After applying the Hadamard gates to the first 4 qubits and initializing the ancillary qubit in |1⟩, what is the quantum state of the system before the oracle is applied? Implement the oracle for the hidden string s = 1011 in Qiskit using CNOT gates. Simulate the circuit and display the final statevector using LaTeX. Equation for Oracle Application: f(x) = s · x mod 2 (11.6.2) Measure the qubits at the end of the circuit and verify whether the measurement results match the hidden string s = 1011

## Setup

In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from IPython.display import display, Latex
import numpy as np

## Reversing the string

## BV

In [19]:
def bernstein_vazirani(hidden_string):
    n=len(hidden_string)
    qc = QuantumCircuit ( n + 1 , n )
 # Step 1: Initialize Hadamard on first n qubits and auxiliary qubit
    qc . h ( range ( n ) ) # Hadamard gates on the first n qubits
    qc . x ( n ) # Initialize auxiliary qubit to |1 >
    qc . h ( n ) # Hadamard gate on the auxiliary qubit

    # Step 2: Apply the oracle ( CNOT gates based on the hidden string )
    for i , bit in enumerate ( hidden_string ) :
        if bit == '1':
            qc . cx (i , n ) # CNOT gate applied when hidden string bit is 1

     # Step 3: Apply Hadamard gates again to the first n qubits ( NOT auxiliary qubit ) 
    qc.h(range(n))

 # Save statevector
    qc.save_statevector()

    return qc , n
    

## Hidden String

In [20]:
hidden_string = "1100" # Expected outcome is the hidden string 110
hidden_string = hidden_string[::-1]
qc , n = bernstein_vazirani( hidden_string )
simulator = AerSimulator ()
compiled_circuit = transpile ( qc , simulator )
result = simulator . run ( compiled_circuit ). result ()

In [21]:
def format_statevector_latex ( statevector , n ) :
    latex_str = r"\ left | \psi \ right \ rangle = "
    basis_states = [ f"|{i :0{ n}b }\\ rangle " for i in range (2** n ) ]
    for amplitude , state in zip ( statevector . data [:2** n ] , basis_states ) :
        real_part = np . real ( amplitude )
        imag_part = np . imag ( amplitude )
        if np .abs ( amplitude ) > 1e-6:
            if imag_part >= 0:
                latex_str += f"({ real_part :.6 f} + { imag_part :.6 f}i){ state }"
            else:
                latex_str += f"({ real_part :.6 f} - { abs ( imag_part ) :.6 f}i){state } + "
    latex_str = latex_str [: -3] # Remove the last ’ + ’
    latex_str += r"$"
    return latex_str

In [22]:
qc.measure(range (n),range (n))
compiled_circuit_with_measure = transpile ( qc , simulator )
result_with_measure = simulator . run ( compiled_circuit_with_measure , shots=1024) . result ()
counts = result_with_measure . get_counts ()
print ( f" Measurement counts : { counts }")

 Measurement counts : {'1100': 1024}
